In [1]:
import sys
 
# setting path
sys.path.append('../')
from citylearn.citylearn import CityLearnEnv
from citylearn_wrapper import CityLearnEnvWrapper
import csv

In [2]:
env_config = {
    "schema": "../data/citylearn_challenge_2022_phase_1/schema.json",
}
num_episodes = 20

In [3]:
env = CityLearnEnvWrapper(env_config)

In [4]:
def collect_dataset(env, num_episodes):
    data = []
    for i in range(num_episodes):
        print(f"Episode {i}, {len(data)} steps collected")
        obs = env.reset()
        done = False
        while not done:
            action = env.action_space.sample()
            next_obs, rew, done, info = env.step(action)
            data.append([obs, next_obs, action.tolist(), rew, done])
            obs = next_obs
    return data

def store_data(data, filename):
    with open(filename, "w") as f:
        writer = csv.writer(f, delimiter='|')
        writer.writerow(["obs", "next_obs", "action", "rew", "done"])
        writer.writerows(data)

In [5]:
data = collect_dataset(env, num_episodes)
store_data(data, "planning_model_data.csv")

Episode 0, 0 steps collected
Episode 1, 8759 steps collected
Episode 2, 17518 steps collected
Episode 3, 26277 steps collected
Episode 4, 35036 steps collected
Episode 5, 43795 steps collected
Episode 6, 52554 steps collected
Episode 7, 61313 steps collected
Episode 8, 70072 steps collected
Episode 9, 78831 steps collected
Episode 10, 87590 steps collected
Episode 11, 96349 steps collected
Episode 12, 105108 steps collected
Episode 13, 113867 steps collected
Episode 14, 122626 steps collected
Episode 15, 131385 steps collected
Episode 16, 140144 steps collected
Episode 17, 148903 steps collected
Episode 18, 157662 steps collected
Episode 19, 166421 steps collected
